# TEST CODE

학번 : 20192xxxxxx

이름 : 전xx

test code입니다 train code에서 학습한 모델을 로드하여
테스트하고 예측을 합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import os
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Now using {device}.")

Now using cuda.


#Testcode에서 데이터셋 파일 경로 수정하는 곳 입니다


In [ ]:
#테스트할 파일 로드
filepath = '/content/drive/MyDrive/ai보안 데이터셋/test-set.csv'

# Test code의 전처리 과정

1. id 컬럼이 존재하면 해당 컬럼을 ids에 저장 후 id 컬럼을 데이터에서 제거
2. id 컬럼이 존재하지 않을시 id가 없다고 None으로 표시
3. Train코드랑 비하게 원핫 인코딩이 필요한 범주형 칼럼들을 나열
4. Train코드에서 저장한 one_hotencode.pkl, scaler.pkl 파일을 로드하여 인코딩 및 스케일러 준비
5. 범주형 변수 제거 및 원핫 인코딩을 한 변수 추가
6. 미리 로드한 sclaer파일로 스케일링
7. X 타입을 float 32으로 바꿔 모델에 입력할 수 있도록 해주고 X_tensor를 통해 전처리된 데이터를 파이토치의 텐서로 변환
8. return_ids가 참이면 id도 같이 반환, 거짓이면 전치리된 데이터만 반환


<비고>

scaler_save 폴더 있어야합니다!

폴더 안에는one_hotencode.pkl, scaler.pkl 파일있어야합니다!

In [ ]:
# 테스트 데이터의 전처리
def test_process_data(filepath,ids_return = False):
    data = pd.read_csv(filepath)

    # 'id' 컬럼 처리
    if 'id' in data.columns:
        ids = data['id'].values
        data = data.drop('id', axis=1)
    else:
        ids = None

    # 범주형 변수들
    categorical_cols = ['proto', 'service', 'state']

    # Train code에서 저장된 onehot_encoder.pkl와 scaler.pkl 로드
    onehot_encoder = joblib.load('/content/drive/MyDrive/scaler_save/onehot_encoder.pkl')
    scaler = joblib.load('/content/drive/MyDrive/scaler_save/scaler.pkl')

    # 원핫 인코딩 범주형 변수들
    X_cat = onehot_encoder.transform(data[categorical_cols])

    # 범주형 컬럼을 제거 및 원핫 인코딩 컬럼 추가
    X = data.drop(columns=categorical_cols)
    X = np.concatenate([X.values, X_cat], axis=1)

    # 스케일링 처리
    X = scaler.transform(X)
    X = X.astype(np.float32)

    # 텐서로 변환
    X_tensor = torch.tensor(X, dtype=torch.float32)

    # id 를 반환할지 말지 결정
    if ids_return:
        return X_tensor, ids
    else:
        return X_tensor

#TEST 함수

예측한 클래스 인덱스 결과를 반환합니다.

In [ ]:
def predic_test(model, test_loader, device):
    model.eval()  # 모델 평가
    predictions = [] # 예측 결과 저장할 리스트

    with torch.no_grad(): # 그래디언계산 방지 및 메모리 사용최적화
        for X_batch in test_loader:
            X_batch = X_batch[0].to(device)  # 배치에서 특성 데이터를 가져와 디바이스로 이동
            outputs = model(X_batch)  # 모델을 통해 출력값 계싼
            _, predicted = torch.max(outputs, 1)  # 예측한 클래스의 가장 큰 값 인덱스를 얻음
            predictions.extend(predicted.cpu().numpy())  # 예측한 값을 넘파이로 반환해 예측 리스트에 추가

    return predictions # 최종 예측결과 반환

#TEST CODE의 모델 정의

Test code에서도 Train code에서 사용한 코드를 정의합니다.

Test code도 train code에서 사용한 모델과 똑같이 정의해야합니다.

In [ ]:
# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN, self).__init__()

        # Conv1D 층
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(128, 256, kernel_size=3, padding=1)

        # 풀링 층
        self.pool = nn.MaxPool1d(2)

        #  완전 연결 층
        self.fc1 = nn.Linear(256 * (input_size // 16), 512)  # input_size // 16은 4개의 풀링 레이어에서 줄어든 크기
        self.fc2 = nn.Linear(512, 256)

        # 출력층
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        # Conv1D 입력을 위한 채널 추가
        x = x.unsqueeze(1)

        # Conv1D + ReLU + 풀링
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)

        # Flatten (배치 크기 및 특징을 펼침)
        x = x.view(x.size(0), -1)

        # 완전 연결층 + ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # 출력층
        x = self.fc3(x)
        return x

# 테스트 데이터 전처리 및 데이터셋 생성

In [ ]:
# 테스트 데이터 처리
X_test, ids = test_process_data(filepath, ids_return = True)
X_test_dataset = TensorDataset(X_test)
X_test_loader = DataLoader(X_test_dataset, batch_size=32, shuffle=False)

# 학습 모델 로드 및 테스트 데이터에 대한 모델 예측

Train code에서 학습한 모델을 로드 및 테스트 데이터에 대한 예측을 수행합니다.

train_save 폴더에서 cnn_mode_epoch.pth를 로드합니다

<비고>

train_save 폴더가 있어야 합니다.

학습한 모델 가중치도 있어야합니다

In [ ]:
#모델 로드
model = CNN(input_size=X_test.shape[1], num_classes=10)  # 출력수는 10개
model.load_state_dict(torch.load('/content/drive/MyDrive/train_save/cnn_model_epoch.pth'))
model.to(device)

#모델 예측
test_predictions = predic_test(model, X_test_loader, device)

<ipython-input-104-0aee29f8801a>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/MyDrive/train_save/cnn_model_epoch.pth'))


# attack_cat의 갯수 확인.

예측한 데이트에서 각각의 공격 카테고리가
얼마나 발생했는지 숫자로 확인합니다.

In [ ]:
#공격유형 카테고리 - 정상 공격 normal를 포함한 9가지 공격유형
attack_categories = ['Normal', 'Fuzzers', 'Analysis',
                     'Backdoor', 'DoS', 'Exploits', 'Generic',
                     'Reconnaissance', 'Shellcode', 'Worms']

# attack_cat별 예측 개수 확인
category_counts = {category: 0 for category in attack_categories}
for pred in test_predictions:
    category = attack_categories[pred]  # attack_cat에대한 예측 매핑
    category_counts[category] += 1

# attack_Cat별 예측 개수 출력
print("attack_cat counts:")
for category, count in category_counts.items():
    print(f'{category}: {count} samples')


attack_cat counts:
Normal: 27750 samples
Fuzzers: 12236 samples
Analysis: 373 samples
Backdoor: 1 samples
DoS: 422 samples
Exploits: 18851 samples
Generic: 18045 samples
Reconnaissance: 4164 samples
Shellcode: 490 samples
Worms: 0 samples


# 예측한 데이터 저장

예측한 데이터를 id와 매핑하여 submisstion 값을 채우며

파일을 저장합니다.

그리고 그 예측한 값을 실제 레이블이 있는 값과 비교하여

잘 예측이 되었는지 정확도를 측정합니다.

In [ ]:
# 예측된 결과를 'id'와 매핑하여 submission 파일 생성
predicted_categories = [attack_categories[pred] for pred in test_predictions]
submission_df = pd.DataFrame({'id': ids, 'attack_cat': predicted_categories})

In [ ]:
# 저장할 경로 지정 파일이름 설정
save_path = '/content/drive/MyDrive/Test_result'
file_name = 'submission.csv'

# 경로가 존재하지 않으면 폴더 생성
os.makedirs(save_path, exist_ok=True)

# 파일 전체 경로
full_path = os.path.join(save_path, file_name)

# 파일 저장
submission_df = pd.DataFrame({'id': ids, 'attack_cat': predicted_categories})
submission_df.to_csv(full_path, index=False)
print(f"Submission saved to {full_path}")


Submission saved to /content/drive/MyDrive/Test_result/submission.csv


In [ ]:
# 실제있는 데이터와 비교
submission_path = '/content/drive/MyDrive/Test_result/submission.csv'
submission = pd.read_csv(submission_path)

true_labels_path = '/content/drive/MyDrive/ai보안 데이터셋/UNSW_NB15_testing-set.csv'
true_labels = pd.read_csv(true_labels_path)


# 'id'와 'attack_cat'을 기준으로 정확도 계산
accuracy = accuracy_score(true_labels['attack_cat'], submission['attack_cat'])
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 75.05%
